# Perform Research with AutoGen and AgentOps

In [1]:
# import libraries

import agentops
from agentops import track_agent
from openai import OpenAI
import os
from dotenv import load_dotenv

import logging

import autogen
from autogen import UserProxyAgent, AssistantAgent

d:\Artificial Synapse Media\AutoGen_Research_Agents\aiagents_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load environment variables

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
AGENTOPS_API_KEY = os.getenv('AGENTOPS_API_KEY')


In [3]:
# log calls assigned to agents

logging.basicConfig(
    level=logging.DEBUG
) 

In [4]:
# initialize agentops
agentops.init(api_key=AGENTOPS_API_KEY, default_tags=["researchPaper-AI-agents"])

🖇 AgentOps:  WARNING: agentops is out of date. Please update with the command: 'pip install --upgrade agentops'
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_session HTTP/11" 200 311
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=a83af8b0-d11c-47ab-a0dc-1271db721de4


In [5]:
# research topic
topic = "Role of AI in Climate Modelling for Environmental Sustainability"

### AutoGen

In [6]:
# set configurations for model

config_list_gpt4 = [{"model": "gpt-4o", "api_key": OPENAI_API_KEY}]

gpt4_config = {
    "cache_seed": 42,  
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120, #in seconds
}

### Create Agents

In [7]:

user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Initialize the chat and pass on the instruction to the ExplainerAgent.",
    code_execution_config=False,
)

explainer = AssistantAgent(
    name="ExplainerAgent",
    llm_config=gpt4_config,
    system_message="""ExplainerAgent. You collect research topic and break it down to subtopics, identifying key research 
                    questions or hypothesis. You also analyze the scope of the research topic and generate suitable 
                    research questions for an academic research. If the research topic is not good enough to generate 
                    enough scope for academic research, rephrase the topics and generate the research questions and hypothesis. 
                    You output the research topic, key research questions and hypothesis generated from the research topic.
                    """,
)

reviewer = AssistantAgent(
    name="ReviewerAgent",
    llm_config=gpt4_config,
    system_message="""ReviewerAgent. You receive the research topic and subtopics from the ExplainerAgent Agent and then gather existing 
                    research, journal articles, papers, and other relevant literature from trusted sources. Summarize 
                    these findings, identify research gaps and organize the information. Produce this summary in the 
                    format of a standard academic literature review, highlighting important existing research and 
                    citing papers used in the review.
                    """,
)

formulator = AssistantAgent(
    name="FormulatorAgent",
    system_message="""FormulatorAgent. You refine the research questions and subtopics from the ExplainerAgent Agent based on the 
                    Literature Review done by the ReviewerAgent and define the exact question the paper will address. If a 
                    hypothesis is required, propose one. Your output is the finalized research question and hypothesis.
                    """,
    llm_config=gpt4_config,
)

director = AssistantAgent(
    name="DirectorAgent",
    llm_config=gpt4_config,
    system_message="""DirectorAgent. You receive the final research question from FormulatorAgent and literature review from 
                    ReviewerAgent and suggest appropriate research methods (qualitative, quantitative, or mixed) based on 
                    the research question. Also, outline data collection methods, sample sizes, tools, and techniques 
                    necessary for this research. Your output is the proposed research methodology, proposing necessary 
                    data collection methods, analysis plans, and tools for the research.
                    """,
)

collector = AssistantAgent(
    name="CollectorAgent",
    llm_config=gpt4_config,
    system_message="""CollectorAgent. You receive the proposed methodology from DirectorAgent Agent and gather real data from the 
                    web or simulate the data related to the research question. Ensure that the data are from verifiable 
                    and reliable sources, and are linked to the research being conducted. Your output are simulated or 
                    recommended datasets for analysis.
                    """,
)

analyst = AssistantAgent(
    name="AnalystAgent",
    system_message="""AnalystAgent. You receive data and analyze the collected data using appropriate statistical or 
                    qualitative analysis methods from CollectorAgent. For quantitative research, you could run statistical tests; for 
                    qualitative research, you would conduct thematic analysis. Your output are charts, graphs, 
                    insights, detected trends and predictions (if there are any). The analysis should be dynamic, 
                    customized to suit the requirement of the research field lies in.
                    """,
    llm_config=gpt4_config,
)

interpreter = AssistantAgent(
    name="InterpreterAgent",
    llm_config=gpt4_config,
    system_message="""InterpreterAgent. You collect data analysis results from AnalystAgent Agent and literature review from 
                    ReviewerAgent. You should interpret the analysis results, compares them with existing research in the 
                    review, and provides insights into how the findings answer the research question or support/refute 
                    the hypothesis, provided by the FormulatorAgent. Your output is a discussion of findings, implications, 
                    and potential limitations.
                    """,
)

recommender = AssistantAgent(
    name="RecommenderAgent",
    llm_config=gpt4_config,
    system_message="""RecommenderAgent. You collect discussion of findings, implications and potential limitations from the 
                    InterpreterAgent Agent and summarize the research findings, proposes future research directions, and 
                    provides recommendations or solutions based on the findings. Your output is conclusion and 
                    recommendation section of the paper.
                    """,
)

formatter = AssistantAgent(
    name="FormatterAgent",
    system_message="""FormatterAgent. You collect different parts of the research paper and ensure the entire research paper 
                    is formatted according to the chosen academic style (APA, MLA, etc.). Generate the fully formatted 
                    research paper using the final research topic from the FormulatorAgent Agent, the literature review 
                    from the ReviewerAgent Agent, the methodology from the DirectorAgent Agent, the data analysis from the 
                    AnalystAgent Agent, the discussion from the InterpreterAgent Agent and the recommendation and conclusion
                    from the RecommenderAgent Agent. You also ensure proper citation process, collecting and formatting 
                    citations from literature sources. Your output is a fully formatted academic paper with citations 
                    and references.
                    """,
    llm_config=gpt4_config,
)

qaengineer = AssistantAgent(
    name="QAEngineerAgent",
    system_message="""QAEngineerAgent. You collect a draft of an academic research paper from FormatterAgent and perform proofreading, plagiarism 
                    checks, grammar and style analysis, and overall coherence evaluation. 
                    """,
    llm_config=gpt4_config,
)

critic = AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gpt4_config,
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, explainer, reviewer, formulator, director, collector, analyst, interpreter, recommender, formatter, qaengineer, critic], messages=[], max_round=3
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='d:\\Artificial Synapse Media\\AutoGen_Research_Agents\\aiagents_venv\\Lib\\site-packages\\certifi\\cacert.pem'
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:httpx:load_ssl_context verify=True cert=None tr

In [8]:
# send a message to the group chat to create the research paper

user_proxy.initiate_chat(
    manager,
    message="""Write an academic research paper on the topic: """ + topic,
)

Admin (to chat_manager):

Write an academic research paper on the topic: Role of AI in Climate Modelling for Environmental Sustainability

--------------------------------------------------------------------------------


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.co

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='d:\\Artificial Synapse Media\\AutoGen_Research_Agents\\aiagents_venv\\Lib\\site-packages\\certifi\\cacert.pem'
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': "You are in a role play game. The following roles are available:\n                Admin: An attentive HUMAN user who can answer questions about the task, and can perform tasks such as running Python code or inputting command line commands at a Linux terminal and repo


Next speaker: ExplainerAgent



DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'ExplainerAgent. You collect research topic and break it down to subtopics, identifying key research \n                    questions or hypothesis. You also analyze the scope of the research topic and generate suitable \n                    research questions for an academic research. If the research topic is not good enough to generate \n                    enough scope for academic research, rephrase the topics and generate the research questions and hypothesis. \n                    You output the research topic, key research questions and hypothesis generated from the research topic.\n                    ', 'role': 'system'}, {'content': 'Write an academic research paper on the topic: Role of AI in Climate Modelling for Environmental Sustainability', 'name': 'Admin', 'role': 'user'}], 'model': 'gpt-4o', 'stream': False, 'temperature': 0}}
D

...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='d:\\Artificial Synapse Media\\AutoGen_Research_Agents\\aiagents_venv\\Lib\\site-packages\\certifi\\cacert.pem'
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': "You are in a role play game. The following roles are available:\n                Admin: An attentive HUMAN user who can answer questions about the task, and can perform tasks such as running Python code or inputting command line commands at a Linux terminal and repo


Next speaker: ReviewerAgent



DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'ReviewerAgent. You receive the research topic and subtopics from the ExplainerAgent Agent and then gather existing \n                    research, journal articles, papers, and other relevant literature from trusted sources. Summarize \n                    these findings, identify research gaps and organize the information. Produce this summary in the \n                    format of a standard academic literature review, highlighting important existing research and \n                    citing papers used in the review.\n                    ', 'role': 'system'}, {'content': 'Write an academic research paper on the topic: Role of AI in Climate Modelling for Environmental Sustainability', 'name': 'Admin', 'role': 'user'}, {'content': "**Research Topic:** The Role of Artificial Intelligence in Climate Modelling for Environmental Sustainability\n\

ReviewerAgent (to chat_manager):

**Literature Review: The Role of Artificial Intelligence in Climate Modelling for Environmental Sustainability**

**1. Overview of Climate Modelling**

Climate modelling is a critical tool in understanding and predicting climate change and its impacts on the environment. Traditional climate models, such as General Circulation Models (GCMs), simulate the Earth's climate system by solving complex mathematical equations that represent physical processes (Randall et al., 2007). These models have been instrumental in projecting future climate scenarios and informing policy decisions. However, they are computationally intensive and often limited by uncertainties in data and model parameters (Flato et al., 2013).

**2. AI Technologies in Climate Science**

Artificial Intelligence (AI) has emerged as a transformative technology in climate science, offering new methods to enhance climate modelling. Machine learning (ML), a subset of AI, is particularly promisin

ChatResult(chat_id=None, chat_history=[{'content': 'Write an academic research paper on the topic: Role of AI in Climate Modelling for Environmental Sustainability', 'role': 'assistant', 'name': 'Admin'}, {'content': "**Research Topic:** The Role of Artificial Intelligence in Climate Modelling for Environmental Sustainability\n\n**Scope of the Research Topic:**\nThe research topic explores the intersection of artificial intelligence (AI) and climate science, focusing on how AI technologies can enhance climate modelling to promote environmental sustainability. This involves examining the capabilities of AI in improving the accuracy, efficiency, and predictive power of climate models, as well as its potential to inform policy-making and sustainable practices.\n\n**Subtopics:**\n1. Overview of Climate Modelling\n2. AI Technologies in Climate Science\n3. Enhancements in Predictive Accuracy and Efficiency\n4. AI's Role in Policy-Making and Sustainable Practices\n5. Challenges and Ethical Co

In [9]:
# end telemetry session
agentops.end_session("Success")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/update_session HTTP/11" 200 139
🖇 AgentOps: Session Stats - Duration: 5m 17.3s | Cost: $0.035380 | LLMs: 8 | Tools: 0 | Actions: 59 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=a83af8b0-d11c-47ab-a0dc-1271db721de4
